In [1]:
import os, json
import torch



In [2]:


train_data_path = "ccks2023_task2_data_release/train.txt"
val_data_path = "ccks2023_task2_data_release/testA.txt"


In [10]:
"""
样本示例：
{"text_id": "402a011912c9ab44aa1ffd4778d10d6b", "text": "北京炼焦化学厂7月1日停产",
"events": [{"type": "停产停业", "entity": "北京炼焦化学厂"}]}


其中events可以为空列表
"""


from IPython.display import display
from pprint import pprint


event_type_set = set()

valid_example_list = []


def generated_sft_data(data_path):

    with open(data_path, 'r', encoding="utf-8") as fin:
        for index, line in enumerate(fin):
            example = json.loads(line)
            events = example["events"]
            if len(events) > 0:
                #     display(index, example)
                #     break
                for event in events:
                    event_type_set.add(event["type"])
                valid_example_list.append(example)
            
            

generated_sft_data(train_data_path)
    



In [11]:
display(event_type_set)

display(len(event_type_set))  # 103个类别

{'业务/资产重组',
 '产品虚假宣传',
 '产品违约/不足额兑付',
 '企业被问询约谈审查',
 '保留意见',
 '信息泄露',
 '债务展期',
 '债务融资失败',
 '债务违约',
 '债务重组',
 '停产停业',
 '停止批准增设分支机构',
 '停止接受新业务',
 '偷税漏税',
 '偿付能力不足',
 '公司停牌',
 '公司注销',
 '公司违规关联交易',
 '公司退市',
 '关闭分支机构',
 '内幕交易',
 '出具虚假证明',
 '分支机构被警方调查',
 '发放贷款出现坏账',
 '吊销业务许可或执照',
 '吊销资质牌照',
 '否定意见',
 '员工罢工示威',
 '基层员工流失',
 '外部信用评级下调',
 '大量投诉',
 '实际控制人变更',
 '实际控制人失联/死亡',
 '实际控制人涉嫌违法',
 '实际控制人涉诉仲裁',
 '实际控制人违规',
 '审计师辞任',
 '履行连带担保责任',
 '延期信息披露',
 '总部被警方调查',
 '扰乱市场秩序',
 '投资亏损',
 '挤兑',
 '授信额度减少',
 '撤销任职资格',
 '无法表示意见',
 '暂停上市',
 '更换基金经理',
 '欠薪',
 '澄清辟谣',
 '盈利能力下降',
 '监管入驻',
 '监管评级下调',
 '盗取隐私信息',
 '破产清算',
 '禁入行业',
 '税务非正常户',
 '窃取别人商业机密',
 '第一大股东变化',
 '签订对赌协议',
 '终身禁入行业',
 '经营亏损',
 '经营期限到期',
 '经营激进',
 '经营资质瑕疵',
 '网站安全漏洞',
 '股东利益斗争',
 '股权冻结/强制转让',
 '股权查封',
 '股权融资失败',
 '股票发行失败',
 '自然灾害',
 '薪酬福利下降',
 '行业排名下降',
 '被列为失信被执行人',
 '被接管',
 '被银行停贷',
 '裁员',
 '警告',
 '评级机构中止评级',
 '误操作',
 '财务信息造假',
 '财务报表更正',
 '责令改正',
 '资产冻结',
 '资产质量下降',
 '资本充足不足',
 '资金紧张',
 '违规催收',
 '连续下跌',
 '退出市场',
 '造假欺诈',
 '重大债务到期',
 '重大安全事故',

103

In [19]:


len(valid_example_list)  # 88860

display(valid_example_list[0])


# 查看单个样本有多个事件的样本数量

num_example_for_multi_event = 0
max_event_count = 0

for example in valid_example_list:
    if len(example["events"]) > 1:
        num_example_for_multi_event += 1
        max_event_count = max(max_event_count, len(example["events"]))
        if max_event_count == 28:
            display(example)
            break


print("包含多个事件的样本：", num_example_for_multi_event, ", ",  "占比：", num_example_for_multi_event / len(valid_example_list) )  # 14754
        
print("单个样本最大的事件数量：", max_event_count)


{'text_id': '4883bb26e4af81a4a562a0cea712e786',
 'text': '同时，为降低管理成本，同日公告显示，陕西黑猫拟吸收合并全资子公司韩城市黑猫化工有限责任公司与韩城市黑猫能源利用有限公司，还注销了另一家全资子公司内蒙古黑猫平旭能源化工有限公司',
 'events': [{'type': '公司注销', 'entity': '韩城市黑猫化工有限责任公司'}]}

{'text_id': 'c6ecdb261848c5a6dc51bca28f190a1f',
 'text': '网上流出的一份#互联网裁员#信息，不知道是否真实，有这些公司的出来说说：2019年互联网公司裁员名单阿里巴巴集团(不开放职位，优酷裁员)爱奇艺(游戏部门裁员)百度(上半年已瘦身，不开放职位)哗哩哗哩(末尾淘汰一定比例，锁HC)贝壳找房(变相裁员)锤子(资金链断裂，已拖欠工资)滴滴(将裁员15%涉及员工超2000人)斗鱼(业务调整，海外直播部门全裁近80人)京东(裁员10%)科大讯飞(变相裁员)美图(裁员)美团(上海点评技术部门裁员50%，持续中)摩拜(美团)(裁员20%30%)美味不用等(裁员60%)OFO(资金链断)平安好房(裁员近300人）趣店(裁员200人）人人车(裁员)苏宁(裁员10%)天天果园(裁员)唯品会(通过绩效考核变相裁员)网易(裁员)小米(金服裁员)星巴克(裁员)有赞(裁员25%)知乎(裁员20%，近300人)36氪(裁员30%)便利蜂(不详)每日优鲜（不详',
 'events': [{'type': '裁员', 'entity': '网易'},
  {'type': '裁员', 'entity': '贝壳找房'},
  {'type': '裁员', 'entity': '平安好房'},
  {'type': '裁员', 'entity': '科大讯飞'},
  {'type': '裁员', 'entity': '摩拜(美团)'},
  {'type': '裁员', 'entity': '爱奇艺'},
  {'type': '资金紧张', 'entity': 'OFO'},
  {'type': '裁员', 'entity': '美图'},
  {'type': '欠薪', 'entity': '锤子'},
  {'type': '资金紧张', 'entity': '锤子'},
  {'type': '裁员', 'entity': '哗哩哗哩'},
  {'type': '裁员', 'entity': '唯品会'},
  {'type': '裁员', 'entity': '天天果园'},
  {'type': '裁员', 'entity': '有赞'},
  {'type': '裁员', 'entity': '京东'},
  {'t

包含多个事件的样本： 11867 ,  占比： 0.13354715282466803
单个样本最大的事件数量： 28


In [35]:

for example in valid_example_list:
    if len(example["events"]) > 1:
        print(example)
        t = []
        for event in example["events"]:
            t.append(event["entity"])
        print(t)
        break


{'text_id': 'ddf32c540de07b119911e55d7036576e', 'text': '化工作室；何炅注销新沂何年何月影视文化工作室；宋茜注销新沂宋茜影视文化工作室；佟丽娅注销新沂大丫影视文化工作室；还有吴谨言、李晨、王千源、胡可、杜淳、陈思成、江疏影、马天宇等注销工作室，以及汪峰、章子怡退出新影佳映（新沂）电影文化发展有限公司股东行列等', 'events': [{'type': '公司注销', 'entity': '何月影视文化工作室'}, {'type': '公司注销', 'entity': '新沂宋茜影视文化工作室'}, {'type': '公司注销', 'entity': '新沂大丫影视文化工作室'}]}
['何月影视文化工作室', '新沂宋茜影视文化工作室', '新沂大丫影视文化工作室']


In [22]:
prompts = [
    "请判断以下文本中的事件类型以及事件类型对应的实体，要求输出所有事件类型，并以JSON格式：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}展示结果。",
    "找出下述文本中的所有事件类型及其对应的实体，按JSON格式：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}输出。",
    "从以下文本中提取事件类型及其实体列表，结果需以JSON格式呈现：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}。",
    "根据下面的文本内容，提取所有事件类型及关联实体，并按照JSON格式：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}输出。",
    "请从下述文本中识别事件类型及其所有关联实体，结果形式要求为JSON：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}。",
    "分析以下文本，提取所有事件类型及其相关联的实体，输出的JSON格式为：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}。",
    "判断下面的内容中出现的事件类型及其对应的实体，按JSON格式输出：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}。",
    "请提取下列文本中的事件类型以及事件类型关联的实体，确保以JSON格式：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}展示结果。",
    "对以下文本内容进行解析，提取事件类型及其实体，结果需以JSON格式：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}。",
    "请从下列文本中找出所有的事件类型及对应的实体，输出结果的JSON格式应为：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}。",
    "分析下面的文本，提取所有涉及的事件类型和关联实体，输出需符合JSON格式：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}。",
    "对给定的文本内容进行事件解析，找出事件类型及其关联的实体，输出格式为JSON：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}。",
    "从以下内容中提取所有的事件类型及其关联的实体，并以JSON格式输出，形式如：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}。",
    "请解析以下文本内容，找出所有事件类型及其实体，并将结果以JSON格式：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}输出。",
    "从下面的文本中识别所有事件类型以及对应的实体列表，输出格式为JSON：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}。",
    "请读取以下文本，找出所有出现的事件类型及关联的实体，按JSON格式：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}列出结果。",
    "判断下述文本中的事件类型及其实体，并以JSON格式输出：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}。",
    "请提取以下内容中的事件类型及其实体列表，确保输出格式为JSON：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}。",
    "根据下面的文本，提取所有事件类型及其关联的实体，并以JSON格式：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}展示。",
    "从下述内容中识别事件类型和对应的实体，按JSON格式输出结果，格式为：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}。"
]
import random
print(random.sample(prompts, 1))

["判断下面的内容中出现的事件类型及其对应的实体，按JSON格式输出：{'事件类型1': ['实体1', '实体2'], '事件类型2': ['实体3', '实体4']}。"]


In [41]:
import random

few_shot_example = """
参考样本：
输入文本：何炅注销新沂何年何月影视文化工作室；宋茜注销新沂宋茜影视文化工作室；佟丽娅注销新沂大丫影视文化工作室；还有吴谨言、李晨、王千源、胡可、杜淳、陈思成、江疏影、马天宇等注销工作室，以及汪峰、章子怡退出新影佳映（新沂）电影文化发展有限公司股东行列等.
输出：{'type': '公司注销', 'entity': ['何月影视文化工作室', '新沂宋茜影视文化工作室', '新沂大丫影视文化工作室']}
"""
def gen_sft_data(example_info_list):

    all_prompt_example_list = []
    
    for example in example_info_list:
        events = example["events"]  # 可能包含多个相同的事件

        # 相同事件的主体entity进行合并
        event_info_dict = {}
        for event in events:
            event_type, event_entity = event["type"], event["entity"]
            if event_type not in event_info_dict:
                event_info_dict[event_type] = []
            event_info_dict[event_type].append(event_entity)
            event_result_info = json.dumps(event_info_dict, ensure_ascii=False)

        system_prompt = random.sample(prompts, 1)[0]
        example_prompt = system_prompt + "\n" + few_shot_example + "\n输入文本：" + example["text"]
        
        # print(example_prompt)
        # print(event_result_info)
        one_example_dict = {"query": example_prompt, "response": event_result_info}
    
        all_prompt_example_list.append(one_example_dict)
    # 保存数据到jsonl

    with open("event_entity_ie.jsonl", "w", encoding="utf-8") as fout:
        for example in all_prompt_example_list:
            fout.write(json.dumps(example, ensure_ascii=False)+"\n")
    
gen_sft_data(valid_example_list)



In [42]:
valid_example_list[0]

{'text_id': '4883bb26e4af81a4a562a0cea712e786',
 'text': '同时，为降低管理成本，同日公告显示，陕西黑猫拟吸收合并全资子公司韩城市黑猫化工有限责任公司与韩城市黑猫能源利用有限公司，还注销了另一家全资子公司内蒙古黑猫平旭能源化工有限公司',
 'events': [{'type': '公司注销', 'entity': '韩城市黑猫化工有限责任公司'}]}

In [ ]:


"""
上诉针对LLM的监督微调是一次性生成完整的答案，也可以分布进行微调：
1、微调文本的事件类型
2、将原始文本与生成的事件类型进行拼接，并生成对应的事件主体


比如：
第一步：
文本：同时，为降低管理成本，同日公告显示，陕西黑猫拟吸收合并全资子公司韩城市黑猫化工有限责任公司与韩城市黑猫能源利用有限公司，还注销了另一家全资子公司内蒙古黑猫平旭能源化工有限公司
输出：事件类型列表：["公司注销"]

第二步：

文本：同时，为降低管理成本，同日公告显示，陕西黑猫拟吸收合并全资子公司韩城市黑猫化工有限责任公司与韩城市黑猫能源利用有限公司，还注销了另一家全资子公司内蒙古黑猫平旭能源化工有限公司
事件类型: 公司注销
事件主体：["韩城市黑猫化工有限责任公司"]
"""



